In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive (5).zip to archive (5).zip


In [ ]:
import os
import re
import zipfile

local_zip = r'archive (5).zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [ ]:
for root,dir,file in os.walk('/tmp'):
  for filename in file:
    print(os.path.join('/tmp',filename))

/tmp/dap_multiplexer.INFO
/tmp/job_applicant_dataset.csv
/tmp/debugger_1z3bxf0h03
/tmp/dap_multiplexer.11856040f2de.root.log.INFO.20250823-125306.111
/tmp/language_service.INFO
/tmp/language_service.11856040f2de.root.log.INFO.20250823-125640.1009
/tmp/colab_runtime.sock
/tmp/__research_colab_dap_multiplexer:dap_multiplexer
/tmp/__research_colab_datalab_kernel_manager_proxy:kernel_manager_proxy
/tmp/__research_colab_datalab_language_service:language_service


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('/tmp/job_applicant_dataset.csv')
print(df.shape)
print(df.columns)
df.head()

(10000, 9)
Index(['Job Applicant Name', 'Age', 'Gender', 'Race', 'Ethnicity', 'Resume',
       'Job Roles', 'Job Description', 'Best Match'],
      dtype='object')


,Job Applicant Name,Age,Gender,Race,Ethnicity,Resume,Job Roles,Job Description,Best Match
0,Daisuke Mori,29,Male,Mongoloid/Asian,Vietnamese,"Proficient in Injury Prevention, Motivation, N...",Fitness Coach,A Fitness Coach is responsible for helping cl...,0
1,Taichi Shimizu,31,Male,Mongoloid/Asian,Filipino,"Proficient in Healthcare, Pharmacology, Medica...",Physician,"Diagnose and treat illnesses, prescribe medica...",0
2,Sarah Martin,46,Female,White/Caucasian,Dutch,"Proficient in Forecasting, Financial Modelling...",Financial Analyst,"As a Financial Analyst, you will be responsibl...",0
3,Keith Hughes,43,Male,Negroid/Black,Caribbean,"Proficient in Budgeting, Supply Chain Optimiza...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1
4,James Davis,49,Male,White/Caucasian,English,"Proficient in Logistics, Negotiation, Procurem...",Supply Chain Manager,A Supply Chain Manager oversees the entire sup...,1


In [ ]:
df.isnull().sum()

,0
Job Applicant Name,0
Age,0
Gender,0
Race,0
Ethnicity,0
Resume,0
Job Roles,0
Job Description,0
Best Match,0


In [ ]:
# Preprocess Resume,Job Description,Job Roles for the training these text

df['tag'] = df['Resume'] + df['Job Description']

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text)
  words = [word for word in words if word not in stop_words]
  lemmatizer = WordNetLemmatizer()
  words = [lemmatizer.lemmatize(word) for word in words]
  preprocessed_word = ' '.join(words)
  return preprocessed_word
df['tag'] = df['tag'].apply(preprocess)
df['Resume'] = df['Resume'].apply(preprocess)
df['Job Description'] = df['Job Description'].apply(preprocess)
print(df['tag'].iloc[1])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


proficient healthcare pharmacology medical terminology patient care surgical skill midlevel experience field hold bachelor degree hold certification advanced cardiovascular life support acls skilled delivering result adapting dynamic environmentsdiagnose treat illness prescribe medication provide ongoing patient care work various specialty including surgery pediatrics internal medicine perform physical exam order test interpret medical result collaborate healthcare provider ensure comprehensive care patient requires medical expertise empathy strong communication skill must stay updated latest medical research treatment


In [ ]:
from sklearn.preprocessing import LabelEncoder,StandardScaler
le = LabelEncoder()
ss = StandardScaler()

df['Ethnicity'] = le.fit_transform(df['Ethnicity'])
df['Job Roles'] = le.fit_transform(df['Job Roles'])
df['Age'] = ss.fit_transform(df[['Age']]).flatten()

In [ ]:
'''df_rec = df['Gender'].astype(str) + " " + \
          df['Ethnicity'].astype(str) + " " + \
          df['Age'].astype(str) + " " + \
          df['tag'].astype(str) + " " + \
          df['Job Roles'].astype(str)'''

df_rec = df['Gender'].astype(str) + " " + \
          df['Ethnicity'].astype(str) + " " + \
          df['Age'].astype(str) + " " + \
          df['Job Roles'].astype(str) + " " + \
          df['tag'].astype(str)
print(df_rec.iloc[1])



Male 6 -1.0105847873465696 34 proficient healthcare pharmacology medical terminology patient care surgical skill midlevel experience field hold bachelor degree hold certification advanced cardiovascular life support acls skilled delivering result adapting dynamic environmentsdiagnose treat illness prescribe medication provide ongoing patient care work various specialty including surgery pediatrics internal medicine perform physical exam order test interpret medical result collaborate healthcare provider ensure comprehensive care patient requires medical expertise empathy strong communication skill must stay updated latest medical research treatment


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
#tfidf = TfidfVectorizer(max_features=5000)
#vectors = tfidf.fit_transform(df_rec).toarray()

cv = CountVectorizer(max_features=5000)
vectors = cv.fit_transform(df_rec).toarray()

X_train,X_test,y_train,y_test = train_test_split(vectors,df['Best Match'],test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


model_1 = XGBClassifier()
model_1.fit(X_train,y_train)
y_pred = model_1.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

model_2= RandomForestClassifier()
model_2.fit(X_train,y_train)
y_pred = model_2.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6375
[[656 367]
 [358 619]]
              precision    recall  f1-score   support

           0       0.65      0.64      0.64      1023
           1       0.63      0.63      0.63       977

    accuracy                           0.64      2000
   macro avg       0.64      0.64      0.64      2000
weighted avg       0.64      0.64      0.64      2000

0.8875
[[915 108]
 [117 860]]
              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1023
           1       0.89      0.88      0.88       977

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000

0.8885
[[917 106]
 [117 860]]
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1023
           1       0.89      0.88      0.89       977

    accuracy                           0.89      2000
   macro avg       0.89      0.89     

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.pipeline import Pipeline

xgb_random = {
    'n_estimators': np.arange(100,500,50),
    'learning_rate' : np.linspace(0.01,1.01,10),
    'max_depth' : np.arange(3,10),
    'subsample' : np.linspace(0.5,1,10)
}

random_search = RandomizedSearchCV(estimator = model_1,param_distributions=xgb_random,n_iter=50,cv=5,verbose=2,random_state=42)
random_search.fit(X_train,y_train)
best_param = random_search.best_params_
print(best_param)

xgb_grid = {
    'n_estimators': [max(50,best_param['n_estimators']-50),best_param['n_estimators']],
    'learning_rate' : [best_param['learning_rate'],min(1.01,best_param['learning_rate']+0.01)],
    'max_depth' : [best_param['max_depth'],min(10,best_param['max_depth']+1)],
    'subsample' : [best_param['subsample']]
}

grid_search = GridSearchCV(estimator = model_1,param_grid=xgb_grid,cv=5,verbose=2)
grid_search.fit(X_train,y_train)
best_param_1 = grid_search.best_params_
print(best_param_1)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END learning_rate=0.1211111111111111, max_depth=6, n_estimators=400, subsample=0.5; total time=   5.9s
[CV] END learning_rate=0.1211111111111111, max_depth=6, n_estimators=400, subsample=0.5; total time=   7.6s
[CV] END learning_rate=0.1211111111111111, max_depth=6, n_estimators=400, subsample=0.5; total time=   6.7s
[CV] END learning_rate=0.1211111111111111, max_depth=6, n_estimators=400, subsample=0.5; total time=   6.8s
[CV] END learning_rate=0.1211111111111111, max_depth=6, n_estimators=400, subsample=0.5; total time=   7.4s
[CV] END learning_rate=1.01, max_depth=7, n_estimators=250, subsample=0.5; total time=   4.8s
[CV] END learning_rate=1.01, max_depth=7, n_estimators=250, subsample=0.5; total time=   4.9s
[CV] END learning_rate=1.01, max_depth=7, n_estimators=250, subsample=0.5; total time=   6.1s
[CV] END learning_rate=1.01, max_depth=7, n_estimators=250, subsample=0.5; total time=   4.8s
[CV] END learning_rate

In [ ]:
model_3 = XGBClassifier(n_estimators=best_param_1['n_estimators'],learning_rate=best_param_1['learning_rate'],max_depth=best_param_1['max_depth'],subsample=best_param_1['subsample'])
model_3.fit(X_train,y_train)
y_pred = model_3.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.891
[[919 104]
 [114 863]]
              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1023
           1       0.89      0.88      0.89       977

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 16.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requi

In [ ]:
from gensim.models import Word2Vec
from gensim.downloader import api
w2v = api.load('word2vec-google-news-300')

data = df['rec'].apply(lambda x: x.split())
model = Word2Vec(data,vector_size=300,window=5,min_count=1,workers=4)


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject